In [1]:
import matplotlib
from src.repository.surebet_repository import SurebetRepository
from src.repository.game_repository import GameRepository
from src.genetic.arbitrage import CrossOverEnum, Population
from src.util import ExportadorDeGraficos, PickleSerializer
from datetime import datetime
repository = SurebetRepository()

In [3]:
from src.model.models import Game
from typing import List

game_repository = GameRepository()
games:  List[Game] = game_repository.get_teams(['c5605d72a41240caa5e15b938ccf6983'])
result = [{game.id: f'{game.home_team} x {game.away_team}'} for game in games]
print(result)


[{'c5605d72a41240caa5e15b938ccf6983': 'Real Madrid x Getafe'}]


<h3> Parâmetros para execução do experimento </h3>

In [2]:
n_generations = 200
n_individuals = 20
mutation_rate = 0.01
crossover_rate = 0.95

In [3]:
def paretos_frontier_plot(front, selected_individuals, path, best_frontier):
    plt = matplotlib.pyplot
    plt.figure()
    # plt.xlim(0, 1.05)
    # plt.ylim(0, 1.05)
    plt.title(f'Best Frontier {best_frontier}º Generation')
    plt.xlabel('Profit')
    plt.ylabel('Dispersation')
    plt.scatter([i.fitness_normalized[0] for i in front], [i.fitness_normalized[1] for i in front])
    plt.scatter([i.fitness_normalized[0] for i in selected_individuals], [i.fitness_normalized[1] for i in selected_individuals], color='green')
    plt.plot([i.fitness_normalized[0] for i in front], [i.fitness_normalized[1] for i in front], color='red', linewidth=1, linestyle='-')
    plt.savefig(path)
    plt.close()


def pick_up_individuals(front):   
    front_fitness = list({tuple(i.fitness) for i in front})  # Remove os itens repetidos
    OPTIMIZE_PROFIT = -1
    OPTIMIZE_MIDDLE = (len(front_fitness) // 2)
    OPTIMIZE_DISPERSATION = 0
    
    front_sorted = sorted(front, key=lambda x: x.fitness[0])
    front_fitness = sorted(front_fitness, key=lambda x: x[0])
    
    aux_individuals_selected = front_fitness[OPTIMIZE_PROFIT], front_fitness[OPTIMIZE_MIDDLE], front_fitness[OPTIMIZE_DISPERSATION]  # Seleciona os 3 indivíduos (Aqui só tem os fitness)
    selected = []
    
    # Método para descobrir qual a é o indivíduo(dentro da fronteira de pareto que possui vários objetos) que possui o fitness escolhido anteriormente
    for i in aux_individuals_selected:
        for individual in front_sorted:
            if list(i) == individual.fitness:
                selected.append(individual)
                break

    return front_sorted, selected

def normalize(front, high_fitness_evaluated):
    max_profit = high_fitness_evaluated[0]
    max_dispersation = high_fitness_evaluated[1]
    for i in front:
        profit_normalized = i.fitness[0] / max_profit
        dispersation_normalized = i.fitness[1] / max_dispersation
        i.fitness_normalized = [profit_normalized, dispersation_normalized]
    

In [4]:
def export_frontier(export: ExportadorDeGraficos, population: Population):
    generation_plot = [0, population.n_generations - 1, population.best_frontier]
    for generation, front in population.pareto_history_front_normalized:
        x = [i[0] for i in front]
        y = [i[1] for i in front]
        if generation in generation_plot:
            export.plot(x, y, generation+1)

def export_solution(population: Population, front_sorted, individuals_selected, export: ExportadorDeGraficos):
    export.hypervolume_plot(population)
    PickleSerializer.save(population, f'{export.data_hora_atual}/pickle.pkl')
    path = export.data_hora_atual + '/' + export.get_file_name(population.best_frontier, 'png')
    export_frontier(export, population)
    paretos_frontier_plot(front_sorted, individuals_selected, path, population.best_frontier)
    
    
def plot_frontier(population: Population, export):
    frontier = population.pareto_history_front[population.best_frontier][1]
    front_sorted, individuals_selected = pick_up_individuals(frontier)
    
    normalize(front_sorted, population.high_fitness_evaluated)
    normalize(individuals_selected, population.high_fitness_evaluated)
    
    export_solution(population, front_sorted, individuals_selected, export)
    
    return individuals_selected

In [5]:
from src.genetic.arbitrage import Individual

def decode_individual(individual: Individual, population: Population, individual_label, report_header, path):
    profit = individual.fitness[0]
    number_of_surebets_chosen = sum([i[0] for i in individual.chromosome])
    surebets_selected = []
    
    for gene, arbitrage in zip(individual.chromosome, individual.arbitrages):
        if gene[0] == 1:
            surebets_selected.append((f'Game: {arbitrage.game_id}', f'Bookmaker 1: {arbitrage.bookmaker_key_OVER}', f'Bookmaker 2: {arbitrage.bookmaker_key_UNDER}', f'Amount bet R$ {round(gene[1] * population.budget, 2)}', f'Profit R$ {((gene[1]*population.budget)*(arbitrage.profit/100)):,.2f}', f'{arbitrage.profit:,.2f}%'))
                        
    percentage_profit = (sum([i[1] for i in individual.chromosome if i[0] == 1]) / number_of_surebets_chosen)
    
    print(f'Number of surebets chosen: {number_of_surebets_chosen}')
    print('Surebets Selected:')
    for i in surebets_selected:
        print(i)
    
    print(f'Total Profit: R$ {profit:,.2f}')
    print('--------------------------------------------------------------------------------------------------------------------------------------------------')
    
    with open(path, 'a') as file:
        if report_header is not None:
            file.write(report_header)

        file.write(f'{individual_label}\n')
        file.write(f'Number of surebets chosen: {number_of_surebets_chosen}\n')
        file.write('Surebets Selected:\n')
        for i in surebets_selected:
            file.write(str(i)+'\n')
        file.write(f'Total Profit: R$ {profit:,.2f}\n')
        file.write('--------------------------------------------------------------------------------------------------------------------------------------------------\n')
        
    

In [6]:
def run_scenario(start_date = '2023-09-02 14:00:00', end_date = '2023-09-02 14:05:00'):
    export = ExportadorDeGraficos()
    
    arbitrages = repository.find_all_unique_between(start_date, end_date)
    population = Population(n_generations=n_generations, n_individuals=n_individuals, mutation_rate=mutation_rate, crossover_rate=crossover_rate, 
                            budget=1000, crossover_strategy=CrossOverEnum.ONE_POINT_CROSSOVER, arbitrages=arbitrages)
    
    individuals_picked_up = plot_frontier(population, export)
    labels = ['Profit Optimization', 'Equilibrium Optimization', 'Dispersation Optimization']
    
    folder = export.data_hora_atual
    start = datetime.strptime(start_date, '%Y-%m-%d %H:%M:%S').strftime('%H:%M')
    end = datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S').strftime('%H:%M')
    date_event = datetime.strptime(start_date, '%Y-%m-%d %H:%M:%S').strftime('%d/%m/%Y')
    path = f"{folder}/report_{start}_{end}.txt"
    
    report_header = f"""*********************************************************** EXPERIMENTAL RESULT REPORT ***********************************************************

[Date]: {date_event}
[Surebet Interval]: {start} to {end}
[Best frontier]: {population.best_frontier}º generation
[Budget]: R$ {population.budget:,.2f}

"""
    for label, individual in zip(labels, individuals_picked_up):
        print(label)
        decode_individual(individual, population, label, report_header, path)
        report_header = None

<h3> Cenários mês de setembro </h3>
<img src="images/surebets_setembro.png" width="75%">
<br> <br>
<img src="images/surebets_02_setembro.png" width="75%">

<h3> Cenário 02/09/2023 </h3>

In [7]:
run_scenario(start_date='2023-09-02 14:00:00', end_date='2023-09-02 14:05:00')

Profit Optimization
Number of surebets chosen: 2
Surebets Selected:
('Game: c5605d72a41240caa5e15b938ccf6983', 'Bookmaker 1: ONEXBET', 'Bookmaker 2: UNIBET_EU', 'Amount bet R$ 1000.0', 'Profit R$ 28.30', '2.83%')
('Game: c5605d72a41240caa5e15b938ccf6983', 'Bookmaker 1: PINNACLE', 'Bookmaker 2: UNIBET_EU', 'Amount bet R$ 0.0', 'Profit R$ 0.00', '0.74%')
Total Profit: R$ 28.30
--------------------------------------------------------------------------------------------------------------------------------------------------
Equilibrium Optimization
Number of surebets chosen: 5
Surebets Selected:
('Game: c5605d72a41240caa5e15b938ccf6983', 'Bookmaker 1: ONEXBET', 'Bookmaker 2: MATCHBOOK', 'Amount bet R$ 150.0', 'Profit R$ 1.12', '0.75%')
('Game: c5605d72a41240caa5e15b938ccf6983', 'Bookmaker 1: MATCHBOOK', 'Bookmaker 2: UNIBET_EU', 'Amount bet R$ 100.0', 'Profit R$ 1.46', '1.46%')
('Game: c5605d72a41240caa5e15b938ccf6983', 'Bookmaker 1: ONEXBET', 'Bookmaker 2: WILLIAMHILL', 'Amount bet R$ 80.0


### Cenário 24/09/2023
